In [ ]:
!wget -O data.tar.gz https://zenodo.org/record/3532678/files/data.tar.gz?download=1
!tar -xzf /content/data.tar.gz >/dev/null

--2023-10-11 23:11:34--  https://zenodo.org/record/3532678/files/data.tar.gz?download=1
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10918523087 (10G) [application/octet-stream]
Saving to: ‘data.tar.gz’

data.tar.gz           0%[                    ]   7.54M   249KB/s    eta 11h 49m

In [ ]:
import os
import torch
import glob
import csv
from tqdm import tqdm
import torch

import matplotlib.pyplot as plt
from torch import nn

from torch.utils.data import random_split, DataLoader, Dataset
import numpy as np
from google.colab import files
import glob
import cv2

import random

torch.manual_seed(12515)
from tqdm import tqdm
from time import time, ctime


In [ ]:
!mkdir data2export

In [ ]:
!rm /content/data2export/trainset.csv
!rm /content/data2export/valset.csv


In [ ]:
!rm -r /content/data2export

In [ ]:
!mkdir data2export

In [ ]:
train_file = open('/content/data2export/trainset.csv', 'a')
val_file = open('/content/data2export/valset.csv', 'a')

In [ ]:
print('Importing data...')
data_files = glob.glob('/content/data/dm_math' + '/*/train.xy')
data_files.remove('/content/data/dm_math/all_modules/train.xy')
for data_file in tqdm(data_files):

  with open(data_file ,'r') as file:
    lines = file.readlines()
    for a in lines[200_000:380_000]:
      train_file.write(a)
    for b in lines[380_000:400_000]:
      val_file.write(b)




In [ ]:
train_size = os.path.getsize('/content/data2export/trainset.csv')
val_size = os.path.getsize('/content/data2export/valset.csv')


In [ ]:
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return f"{num:.1f} {unit}{suffix}"
        num /= 1024.0
    return f"{num:.1f} Yi{suffix}"

file_size_readable_train = sizeof_fmt(train_size, 'Gi')
file_size_readable_val = sizeof_fmt(val_size, 'Gi')

print('Memoria occupata dal train file: ', file_size_readable_train)
print('Memoria occupata dal val file: ', file_size_readable_val)


In [ ]:
!mkdir /content/data2export/interpolate
!mkdir /content/data2export/extrapolate

In [ ]:
# Use glob to get a list of paths that match the pattern
data_files = glob.glob('/content/data/dm_math' + '/*/interpolate.xy')
data_files.remove('/content/data/dm_math/all_modules/interpolate.xy')
tasks_interpolate = []
for data_file in data_files:
  task_name = data_file.split('/')[-2]
  tasks_interpolate.append(task_name)

  task_file = open('/content/data2export/interpolate/' + task_name +'.csv', 'a')
  with open(data_file, 'r') as file:
    lines = file.readlines()
    for a in lines:
      task_file.write(a)



In [ ]:
print(tasks_interpolate)

In [ ]:


# Use glob to get a list of paths that match the pattern
data_files = glob.glob('/content/data/dm_math' + '/*/extrapolate.xy')
data_files.remove('/content/data/dm_math/all_modules/extrapolate.xy')
tasks_extrapolate = []
for data_file in data_files:
  task_name = data_file.split('/')[-2]

  tasks_extrapolate.append(task_name)

  task_file = open('/content/data2export/extrapolate/' + task_name +'.csv', 'a')

  with open(data_file, 'r') as file:
    lines = file.readlines()
    for a in lines:
      task_file.write(a)


In [ ]:
print(tasks_extrapolate)

In [ ]:
!tar -czvf math_data_200k.tar.gz /content/data2export

In [ ]:
!cp /content/math_data_200k.tar.gz /content/drive/MyDrive/DeepLproject_2023/

In [ ]:
from pickle import NONE
class MathData(Dataset):
  def __init__(self, dir_path, partition, task=None, max_len = 200):
    self.dir_path = dir_path

    self.data = []
    self.max_len = max_len
    self.pad_char =  '˽'
    self.prefix_char = '$'
    self.suffix_char = '§'
    self.digit2idx = {'˽': 0, '$': 1, '§': 2, 'w': 3, 'h': 4, 'a': 5, 't': 6, ' ': 7, 'i': 8, 's': 9, 'p': 10, 'r': 11, 'o': 12, 'b': 13, 'f': 14, 'e': 15, 'q': 16, 'u': 17, 'n': 18, 'c': 19, 'l': 20, 'k': 21, 'd': 22, 'm': 23, '?': 24, '0': 25, '{': 26, 'g': 27, ':': 28, '3': 29, ',': 30, '1': 31, '7': 32, '}': 33, '.': 34, 'v': 35, '/': 36, '4': 37, 'x': 38, '5': 39, '2': 40, '9': 41, '8': 42, 'z': 43, 'y': 44, '6': 45, 'j': 46, '-': 47, '*': 48, '+': 49, '=': 50, '(': 51, ')': 52, "'": 53, '>': 54, '<': 55, '!': 56}
    self.idx2digit = {v : k for k,v in self.digit2idx.items()}
    self.tasks_interpolate = ['probability__swr_p_sequence', 'probability__swr_p_level_set', 'numbers__lcm_composed', 'calculus__differentiate_composed', 'numbers__lcm', 'algebra__linear_1d', 'arithmetic__add_sub_multiple', 'polynomials__simplify_power', 'measurement__conversion', 'polynomials__compose', 'numbers__base_conversion', 'polynomials__evaluate', 'arithmetic__mul', 'algebra__linear_1d_composed', 'arithmetic__add_or_sub', 'numbers__place_value_composed', 'polynomials__add', 'numbers__list_prime_factors_composed', 'arithmetic__mul_div_multiple', 'measurement__time', 'numbers__div_remainder', 'arithmetic__simplify_surd', 'numbers__place_value', 'comparison__closest_composed', 'polynomials__expand', 'numbers__is_factor', 'numbers__div_remainder_composed', 'arithmetic__nearest_integer_root', 'numbers__round_number_composed', 'arithmetic__mixed', 'algebra__polynomial_roots_composed', 'algebra__sequence_nth_term', 'arithmetic__add_or_sub_in_base', 'comparison__pair_composed', 'algebra__linear_2d', 'polynomials__coefficient_named', 'comparison__kth_biggest', 'comparison__sort', 'polynomials__collect', 'numbers__list_prime_factors', 'algebra__polynomial_roots', 'numbers__round_number', 'numbers__is_prime', 'algebra__sequence_next_term', 'comparison__closest', 'comparison__sort_composed', 'numbers__gcd', 'arithmetic__div', 'numbers__gcd_composed', 'calculus__differentiate', 'numbers__is_prime_composed', 'comparison__pair', 'numbers__is_factor_composed', 'polynomials__evaluate_composed', 'algebra__linear_2d_composed', 'comparison__kth_biggest_composed']
    self.tasks_extrapolate = ['probability__swr_p_level_set_more_samples', 'measurement__conversion', 'arithmetic__mixed_longer', 'arithmetic__mul_big', 'arithmetic__mul_div_multiple_longer', 'comparison__sort_more', 'arithmetic__add_or_sub_big', 'comparison__closest_more', 'comparison__kth_biggest_more', 'arithmetic__div_big', 'arithmetic__add_sub_multiple_longer', 'numbers__round_number_big', 'probability__swr_p_sequence_more_samples', 'numbers__place_value_big', 'algebra__polynomial_roots_big']

    print('Loarding', partition, 'data...')
    self.load_data(dir_path, partition, task)

  def load_data(self, dir_path, partition, task = None): #partition choose between ('train', 'val')
    if partition == 'train':
      data_path = os.path.join(dir_path, 'trainset.csv')

    elif partition == 'val':
      data_path = os.path.join(dir_path, 'valset.csv')

    elif partition == 'interpolate':
      if isinstance(task, (int)):
        data_path = os.path.join(dir_path, 'interpolate', self.tasks_interpolate[task] + '.csv')
      else:
        data_path = os.path.join(dir_path, 'interpolate', task + '.csv')

    elif partition =='extrapolate':
      if isinstance(task, (int)):
        data_path = os.path.join(dir_path, 'extrapolate', self.tasks_extrapolate[task] +'.csv')
      else:
        data_path = os.path.join(dir_path, 'extrapolate', task + '.csv')

    else:
      raise Exception('Please choose the partition between [train, val, interpolate, extrapolate]')

    with open(data_path, 'r') as file:
      lines = file.readlines()
      for line in tqdm(lines):
        self.data.append(line)

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):

    line = self.data[idx].split('\t')

    x_digits = [self.digit2idx[self.prefix_char]] + [self.digit2idx[digit.lower()] for digit in line[0]] + [self.digit2idx[self.suffix_char]] + [self.digit2idx[self.pad_char]]*(self.max_len-3-len(line[0]))

    y_digits = [self.digit2idx[self.prefix_char]] + [self.digit2idx[digit.lower()] for digit in line[1].replace('\n', '')] + [self.digit2idx[self.suffix_char]] + [self.digit2idx[self.pad_char]]*(self.max_len-3-len(line[1].replace('\n', '')))

    x_ids = torch.LongTensor(x_digits)
    y_ids = torch.LongTensor(y_digits)
    return x_ids, y_ids


In [ ]:
wow = MathData('/content/data2export', 'val')

In [ ]:
bebe = MathData('/content/data2export', 'interpolate', 4)

In [ ]:
bebe[533]